In [1]:

# -*- coding: utf-8 -*-
"""
Created on Wed Jan 13 17:11:27 2021

@author: User
"""

import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer

class DBUpdater:  
    def __init__(self):
        """생성자: MySQLDB 연결 및 종목코드 딕셔너리 생성"""
        # self내의 .conn의 변수를 설정, 연결만 하는거지 cursor를 만들지 않으면 건들수 없음
        self.conn = pymysql.connect(host='localhost', user='root',password='qet[]zc//!@92', db='INVESTAR', charset='utf8')  

        with self.conn.cursor() as curs:  # self.conn의 cousor을 가져오겠다는 것 # with : for문처럼 밑에 작업들이 다 되기전까지 끝내지 않음
           # DB를 건들수 있는 cursor를 만들겠다는것
            sql = """ 
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            # company_info라는 TABLE을 만들겠다는 것
            # PRIMARY KEY : recode단위로 unique하게 선별할 수 있게해줌 => 종목코드로 PK를 지정하겠다는 것
            # ID Number를 통해 개인이 unique하게 식별하는 것과 같은 느낌
            curs.execute(sql)    

            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            # daily_price는 종목코드뿐만아니라 날짜로도 구별되야 하기 때문에 PK를 code와 date 2개로 둠
            
            curs.execute(sql) # 위에 작성한 코드를 실제로 보내겠다는(전송) 것 ; cursor는 가리키기만 할 뿐(연결만 되어있다는 것, 지시자)
                            # execute ; workbench에서 구동시키는 것처럼 위 sql구문을 실행시킨다는 것
        self.conn.commit() # execute는 실행만 시킨것 commit를 해줘야 DB로 밀어넣음
        
        self.codes = dict() # codes라는 변수를 dict()로 만들었다는 것
                
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 

    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]   
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx

    def update_comp_info(self): # DB에 업데이트 하겠다는 것
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn) # 처음에 conpany_info에 data가 없으므로 df는 null
                                        # pandas에서 sql을 읽는 법 ; read_sql 그냥 알아놓자
        for idx in range(len(df)): # 처음에는 건너뜀 null값이라 len이 0
            self.codes[df['code'].values[idx]] = df['company'].values[idx]

        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone() # companuy_info의 max(last_update) 라는 정보를 rs에 저장
            today = datetime.today().strftime('%Y-%m-%d') # 자동으로 오늘 날짜를 가져오는 것
            
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today: # 업데이트된 날짜가 어제인지를 물어본다는 것
                krx = self.read_krx_code()
                for idx in range(len(krx)): #krx len은 2000얼마임
                    code = krx.code.values[idx] 
                    company = krx.company.values[idx] 
                    
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                     # REPLACE INTO (PK로 비교); 있으면 REPLACE 없으면 UPDATE
                    curs.execute(sql) # 바로 위 sql구문을 실행
                    
                    self.codes[code] = company # codes 위에 만든 비어있는 dict => key : value로 누적시킨다는 것
                    
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                    # 어느정도 진행되는지 찍어내는 것
                self.conn.commit() # DB에 밀어넣음
                print('')          
                
    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url, headers={'User-agent' : 'Mozilla/5.0'}).text, 'lxml' )
            pgrr = html.find("td", class_="pgRR")

            if pgrr is None :
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1]
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(requests.get(pg_url,
                                                        headers={'User-agent' : 'Mozilla/5.0'}).text)[0])    
                
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, company, code, page, pages), end="\r")

            df = df.rename(columns={'날짜':'date', '종가':'close', '전일비':'diff', '시가':'open',
                                   '고가':'high', '저가':'low', '거래량':'volume'})  
            df['date'] = df['date'].replace('.','-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'close', 'diff', 'open', 'high', 'low', 'volume']]


        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                    'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                    ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])     

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 70
                config = {'pages_to_fetch' : pages_to_fetch}
                json.dump(config, out_file)
                
        self.update_daily_price(pages_to_fetch)
        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                                  hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                                 minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                                 second=0)
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ...".format(tmnext.strftime('%Y-%m-%d %H:%M')))
        t.start()

if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

[2021-06-22 14:08] #0001 REPLACE INTO company_info VALUES (004840, DRB동일, 2021-06-22)
[2021-06-22 14:08] #0002 REPLACE INTO company_info VALUES (155660, DSR, 2021-06-22)
[2021-06-22 14:08] #0003 REPLACE INTO company_info VALUES (001250, GS글로벌, 2021-06-22)
[2021-06-22 14:08] #0004 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2021-06-22)
[2021-06-22 14:08] #0005 REPLACE INTO company_info VALUES (092220, KEC, 2021-06-22)
[2021-06-22 14:08] #0006 REPLACE INTO company_info VALUES (016380, KG동부제철, 2021-06-22)
[2021-06-22 14:08] #0007 REPLACE INTO company_info VALUES (001390, KG케미칼, 2021-06-22)
[2021-06-22 14:08] #0008 REPLACE INTO company_info VALUES (058860, KTis, 2021-06-22)
[2021-06-22 14:08] #0009 REPLACE INTO company_info VALUES (011070, LG이노텍, 2021-06-22)
[2021-06-22 14:08] #0010 REPLACE INTO company_info VALUES (066570, LG전자, 2021-06-22)
[2021-06-22 14:08] #0011 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2021-06-22)
[2021-06-22 14:08] #0012 REPLACE INTO company_info VAL